In [1]:
import os
import re
import csv
import sage
import pandas as pd
import numpy as np
from multiprocessing import Pool, cpu_count
from tqdm import tqdm, tqdm_notebook
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

tqdm.pandas(tqdm_notebook)
data_path = "/home/dfsnow/rbans/data/"

In [2]:
# Create ekphrasis preprocessor class
ekphrasis_processor = TextPreProcessor(
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user', 'time', 'date', 'number'],  # normalize terms
    fix_html=True,  # fix HTML tokens  
    segmenter="english",  # corpus for word segmentation
    corrector="english",  # corpus for spell correction
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # unpack contractions 
    spell_correct_elong=False,  # spell correction for elongated words
    dicts=[emoticons]  # replace emojis with words
)

ekphrasis_tokenizer = SocialTokenizer(lowercase=True).tokenize
flatten = lambda l: [item for sublist in l for item in sublist]

Reading english - 1grams ...
Reading english - 2grams ...
Reading english - 1grams ...


## Shuffling

This section is dedicated to ensuring that the sample drawn from Postgres is sufficiently shuffled.

In [ ]:
# Load the shuffled main training data into memory
train = pd.read_csv(
    os.path.join(data_path, "main_data_sample.csv"),
    dtype={"":np.int64, "id": str, "score": str, "body": str, "label": int}
)

In [ ]:
# Shuffle all the training data to ensure a random distribution
train = shuffle(train)

In [ ]:
# Save the shuffled data to disk
train.to_csv(os.path.join(data_path, "main_data_shuffled.csv"), quoting=csv.QUOTE_NONNUMERIC)

## Preprocessing

This section is dedicated to preprocessing all of the data using ekphrasis. Doing this in chunks is more efficient, since ekphrasis takes quite awhile to run. After processing all chunks, we concatenate them back together in the command line and load the cleaned data from here on.

In [ ]:
def read_iter(start_row):
    chunksize = 100000
    filename = int(start_row / chunksize)
    shuffled_data = os.path.join(data_path, "main_data_shuffled.csv")
    chunk = pd.read_csv(
        shuffled_data, index_col=0, skiprows=start_row, nrows=chunksize,
        names=["index", "id", "score", "body", "label"],
        dtype={"index": np.int64, "id": str, "score": str, "body": str, "label": int},
    )
    chunk["body"] = chunk.body.map(ekphrasis_processor.pre_process_doc)
    chunk.to_csv(
        os.path.join(data_path, "split/" + str(filename).zfill(4) + "_preprocessed_chunk.csv"),
        quoting=csv.QUOTE_NONNUMERIC,
        header=False, index=False
    )

train_len = 229603257
completed = [int(chunk[0:4]) * 100000 for chunk in os.listdir(os.path.join(data_path, 'split/'))] 
all_jobs = list(range(0, train_len, 100000))
jobs = set(all_jobs) - set(completed)
print(jobs)
pool = Pool(cpu_count() - 1)
pool.map(read_iter, jobs)

{113600000, 88000000, 11200000, 36800000, 62400000, 139200000, 164800000, 190400000, 6900000, 82200000, 210200000, 184600000, 159000000, 110700000, 85100000, 136300000, 33900000, 8300000, 59500000, 161900000, 187500000, 213100000}


## SAGE Filtering

This section is dedicated to removing noise from our labelled posts. By using Sparse Additive Generative Model (SAGE), we can find words that are more common in hateful posts than in nonhateful posts. These words can be used as an initial filter for our training data by removing any "hateful" posts that do not container them. As a result, we filter out meaningless or nonspecific labelled posts and keep only the more specific language of hate.

In [ ]:
def tokenize(body):
    tokenized = flatten(ekphrasis_tokenizer(str(body)))
    return tokenized

def get_vocab(start_row):
    chunksize = 100000
    filename = int(start_row / chunksize)
    preprocessed_data = os.path.join(data_path, "main_data_preprocessed.csv")
    chunk = pd.read_csv(
        preprocessed_data, skiprows=start_row, nrows=chunksize,
        names=["id", "score", "body", "label"],
        dtype={"id": str, "score": str, "body": str, "label": int},
    )
    
    hate = chunk.loc[train.label == 1].body.map(tokenizer)
    nonhate = chunk.loc[train.label == 1].body.map(tokenizer)
    
    hate.to_pickle(os.path.join(data_path, "vocab/" + str(filename).zfill(4) + "_hate.csv"))
    nonhate.to_pickle(os.path.join(data_path, "vocab/" + str(filename).zfill(4) + "_nonhate.csv"))

train_len = 229603257
completed = [int(chunk[0:4]) * 100000 for chunk in os.listdir(os.path.join(data_path, 'vocab/'))] 
all_jobs = list(range(0, train_len, 100000))
jobs = set(all_jobs) - set(completed)
print(jobs)
pool = Pool(cpu_count() - 1)
pool.map(get_vocab, jobs)

{0, 25600000, 51200000, 102400000, 128000000, 76800000, 153600000, 179200000, 204800000, 2900000, 28500000, 54100000, 79700000, 105300000, 130900000, 156500000, 182100000, 207700000, 5800000, 31400000, 57000000, 82600000, 108200000, 133800000, 159400000, 185000000, 210600000, 8700000, 34300000, 59900000, 85500000, 111100000, 136700000, 162300000, 187900000, 213500000, 11600000, 37200000, 62800000, 88400000, 114000000, 139600000, 165200000, 190800000, 216400000, 14500000, 40100000, 91300000, 116900000, 65700000, 142500000, 168100000, 193700000, 219300000, 17400000, 43000000, 94200000, 119800000, 68600000, 145400000, 171000000, 196600000, 222200000, 20300000, 45900000, 97100000, 122700000, 71500000, 148300000, 173900000, 199500000, 225100000, 23200000, 48800000, 125600000, 100000000, 74400000, 151200000, 176800000, 202400000, 228000000, 500000, 26100000, 51700000, 77300000, 102900000, 128500000, 154100000, 179700000, 205300000, 29000000, 3400000, 54600000, 105800000, 80200000, 131400000,

In [40]:
# Load in the shuffled, preprocessed training data
train = pd.read_csv(
    os.path.join(data_path, "main_data_preprocessed.csv"),
    dtype={"id": str, "score": str, "body": str, "label": int},
    nrows=50000000
)

In [ ]:
# Get word counts for the hate and non-hate samples, tokenizing with ekphrasis
base_counts = dict(Counter(flatten([ekphrasis_tokenizer(str(body)) for body in train.loc[train.label == 0].body])))
hate_counts = dict(Counter(flatten([ekphrasis_tokenizer(str(body)) for body in train.loc[train.label == 1].body])))

In [ ]:
# Get the most common hate words
hate_vocab = [word for word,count in Counter(hate_counts).most_common(50000)]

In [ ]:
# Convert the counts into numerically comparable arrays
hate_array = np.array([hate_counts.get(word, 0) for word in hate_vocab])
base_array = np.array([base_counts.get(word, 0) for word in hate_vocab]) + 1.

In [ ]:
# Use the SAGE algorithm to get the top K words from hate subs
mu = np.log(base_array) - np.log(base_array.sum())
beta = sage.estimate(hate_array, mu)
hate_words = sage.topK(beta, hate_vocab, 2000)
print(hate_words)

In [ ]:
# Save the hate words to a list CSV
with open(os.path.join(data_path, "hate_words.csv"), "w") as f:
    writer = csv.writer(f)
    for word in hate_words:
        writer.writerow([word])

In [ ]:
# Number of hateful posts prefiltering
train.label.sum()

In [43]:
# Filter out the chaff posts from hateful posts using SAGE
train = train.loc[
    (train.label == 1 & train.body.str.contains('|'.join(re.escape(w) for w in hate_words), case=False)) |
    (train.label == 0)]

train.label.sum()

KeyboardInterrupt: 

## Splitting

Splitting the data into train, test, and validate using an 80, 20, 20 split

In [ ]:
# Split hate and nonhate datasets into train, test, and validate
train, test = train_test_split(train, test_size=0.2, random_state=2) 
train, validate = train_test_split(train, test_size=0.2, random_state=2) 

In [ ]:
# Save train test and validate to CSV
train.to_csv(os.path.join(data_path, "main_train.csv"), index=False)
test.to_csv(os.path.join(data_path, "main_train.csv"), index=False)
validate.to_csv(os.path.join(data_path, "main_train.csv"), index=False)